# Project: Wrangling and Analyze Data

In [62]:
# Import libraries
import os
import re
import tweepy
import config
import requests
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

sn.set()

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [2]:
# Load twitter-archive-enhaced.csv into a pandas dataframe
archive = pd.read_csv('twitter-archive-enhanced.csv')
archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [3]:
# Get image-predictions.tsv and write to file image_predictions.csv to local folder
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
file = 'image_predictions.csv'
try:
    if file not in os.listdir():
        with open('image_predictions.tsv', mode='wb') as f:
            response = requests.get(url)
            f.write(response.content)
except Exception as e:
    print(e)
predictions = pd.read_csv('image_predictions.tsv', delimiter='\t')
predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [4]:
# Instantiate client object and query tweet information
client = tweepy.Client(bearer_token=config.bearer_token)

In [5]:
# Get tweets
length = len(archive)
tweet_list = []
count = 0
try:
    for i in range(0, length, 100):
        tweets = client.get_tweets(ids=[str(i) for i in archive.tweet_id.iloc[i:i+100]],
                                       tweet_fields=['created_at','public_metrics'])
        for tweet in tweets.data:
            tweet_json = {
                'tweet_id': tweet.id,
                'timestamp': tweet.created_at,
                'retweet_count': tweet.public_metrics['retweet_count'],
                'reply_count': tweet.public_metrics['reply_count'],
                'like_count': tweet.public_metrics['like_count']
            }
            tweet_list.append(tweet_json)
except Exception as e:
    print(e)

In [6]:
# See tweet_json dataframe
tweet_json = pd.DataFrame(tweet_list)
tweet_json.head()

,tweet_id,timestamp,retweet_count,reply_count,like_count
0,892420643555336193,2017-08-01 16:23:56+00:00,6969,139,33696
1,892177421306343426,2017-08-01 00:17:27+00:00,5272,170,29220
2,891815181378084864,2017-07-31 00:18:03+00:00,3464,110,21976
3,891689557279858688,2017-07-30 15:58:51+00:00,7191,142,36785
4,891327558926688256,2017-07-29 16:00:24+00:00,7715,193,35178


## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



In [7]:
# See archive observations
archive.shape

(2356, 17)

In [8]:
# See archive summary info
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [9]:
# See predictions summary info
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [10]:
# See tweet_json summary info
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2327 entries, 0 to 2326
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   tweet_id       2327 non-null   int64              
 1   timestamp      2327 non-null   datetime64[ns, UTC]
 2   retweet_count  2327 non-null   int64              
 3   reply_count    2327 non-null   int64              
 4   like_count     2327 non-null   int64              
dtypes: datetime64[ns, UTC](1), int64(4)
memory usage: 91.0 KB


In [11]:
# Check archive random sample
archive.sample(20)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
121,869596645499047938,NaN,NaN,2017-05-30 16:49:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Scout. He just graduated. Officially a...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/869596645...,12,10,Scout,doggo,None,None,None
102,872620804844003328,NaN,NaN,2017-06-08 01:06:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Monkey. She's supporting owners everyw...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/872620804...,13,10,Monkey,None,None,None,None
2207,668627278264475648,NaN,NaN,2015-11-23 03:09:00 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Timofy. He's a pilot for Southwest. It...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668627278...,9,10,Timofy,None,None,None,None
350,831552930092285952,NaN,NaN,2017-02-14 17:17:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Dutch. He dressed up as his favorite e...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/831552930...,13,10,Dutch,None,None,None,None
42,884247878851493888,NaN,NaN,2017-07-10 03:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",OMG HE DIDN'T MEAN TO HE WAS JUST TRYING A LIT...,NaN,NaN,NaN,https://twitter.com/kaijohnson_19/status/88396...,13,10,None,None,None,None,None
949,752309394570878976,NaN,NaN,2016-07-11 01:11:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Everyone needs to watch this. 1...,6.753544e+17,4.196984e+09,2015-12-11 16:40:19 +0000,https://twitter.com/dog_rates/status/675354435...,13,10,None,None,None,None,None
2326,666411507551481857,NaN,NaN,2015-11-17 00:24:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is quite the dog. Gets really excited whe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666411507...,2,10,quite,None,None,None,None
1696,681242418453299201,NaN,NaN,2015-12-27 22:37:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Champ. He's being sacrificed to the Az...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/681242418...,10,10,Champ,None,None,None,None
1026,746131877086527488,NaN,NaN,2016-06-24 00:04:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Gustav. He has claimed that plant. It ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/746131877...,10,10,Gustav,None,None,None,None
512,811744202451197953,NaN,NaN,2016-12-22 01:24:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Halo. She likes watermelon. 13/10 http...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/811744202...,13,10,Halo,None,None,None,None


In [12]:
# Check predictions random sample
predictions.sample(20)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
461,674800520222154752,https://pbs.twimg.com/media/CV1ffl3XAAAiFyr.jpg,1,Pembroke,0.876479,True,Cardigan,0.096911,True,dingo,9.195670e-03,False
1223,744709971296780288,https://pbs.twimg.com/media/ClW9w7mWEAEFN1k.jpg,1,Shetland_sheepdog,0.234431,True,Samoyed,0.114876,True,collie,8.661370e-02,True
408,673711475735838725,https://pbs.twimg.com/media/CVmA_osW4AArAU1.jpg,1,Maltese_dog,0.607401,True,toy_poodle,0.143836,True,Sealyham_terrier,6.390700e-02,True
575,678643457146150913,https://pbs.twimg.com/media/CWsGnyMVEAAM1Y1.jpg,1,Labrador_retriever,0.338757,True,flat-coated_retriever,0.304470,True,chest,9.339230e-02,False
884,698953797952008193,https://pbs.twimg.com/media/CbMuxV5WEAAIBjy.jpg,1,Italian_greyhound,0.382378,True,redbone,0.102255,True,shower_cap,7.683370e-02,False
1545,792050063153438720,https://pbs.twimg.com/media/Cv3tU38WcAASFas.jpg,2,komondor,0.942856,True,swab,0.052715,False,Tibetan_terrier,2.743000e-03,True
1911,853639147608842240,https://pbs.twimg.com/media/C9i8RhhXoAAdkMT.jpg,1,German_shepherd,0.509879,True,malinois,0.237311,True,kelpie,4.691620e-02,True
1674,813142292504645637,https://pbs.twimg.com/media/C0jcmOKVQAAd0VR.jpg,3,beagle,0.848735,True,Ibizan_hound,0.044602,True,Italian_greyhound,1.861080e-02,True
475,675149409102012420,https://pbs.twimg.com/media/CV6czeEWEAEdChp.jpg,1,chow,0.999876,True,Tibetan_mastiff,0.000059,True,Tibetan_terrier,2.877850e-05,True
349,672482722825261057,https://pbs.twimg.com/media/CVUjd14W4AE8tvO.jpg,1,West_Highland_white_terrier,0.586173,True,borzoi,0.206620,True,Great_Pyrenees,6.065270e-02,True


In [13]:
# Check for duplicates in archive
archive.duplicated().sum()

0

In [14]:
# Check for duplicates in predictions
predictions.duplicated().sum()

0

In [15]:
# Check for duplicates in tweet_json
tweet_json.duplicated().sum()

0

In [16]:
# See Summary stats for meaningful numeric columns
archive.drop(columns=[i for i in list(archive) if '_id' in i]).describe()

,rating_numerator,rating_denominator
count,2356.000000,2356.000000
mean,13.126486,10.455433
std,45.876648,6.745237
min,0.000000,0.000000
25%,10.000000,10.000000
50%,11.000000,10.000000
75%,12.000000,10.000000
max,1776.000000,170.000000


In [17]:
# See summary stats for tweet_json
tweet_json.drop('tweet_id', axis=1).describe()

,retweet_count,reply_count,like_count
count,2327.000000,2327.000000,2327.000000
mean,2458.665664,42.122475,7026.565965
std,4163.685641,70.699083,10919.547809
min,1.000000,0.000000,0.000000
25%,492.500000,9.000000,1220.500000
50%,1144.000000,23.000000,3037.000000
75%,2844.500000,48.000000,8564.000000
max,70330.000000,1120.000000,144245.000000


### Quality issues

1. `timestamp` column datatype is `object` in `archive`.

2. Both `in_reply` columns have only 78 non-null values in `archive`.

3. All `retweeted` colums have only 181 non-null values in `archive`.

4. Dog name of row `275` is  `10` in `archive` and others are random letters or words(Visual asessment).

5. Some dog types are missing even though are contained in the `text` and others are wrong.

6. The source column values contain `HTML` tags and other irrelevant information.

7. `expanded_urls` is not human friendly.

8. `tweet id` column values are of type `int` in all dataframes.

9. `image_predictions` dataframe has 2075 observations.

10. `tweet_json` dataframe has 2327 observations. 

11. `rating_numerator` has a maximum value of 1776.

### Tidiness issues
1. `doggo`, `floofer`, `pupper`, `puppo` should be observations and not columns(Visual Assesment).

2. `tweet_json` should be part of `archive`(Visual Assesment).

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [18]:
# Make copies of original dataframes
archive_clean = archive.copy()
predictions_clean = predictions.copy()
tweet_json_clean = tweet_json.copy()

### Issue #1:

#### Define:

- Change the `timestamp` column datatype in `archive` to `datetime`.

#### Code

In [19]:
# Changing timestamp col datatype
archive_clean['timestamp'] = pd.to_datetime(archive_clean.timestamp)

#### Test

In [20]:
# See changes 
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   int64              
 1   in_reply_to_status_id       78 non-null     float64            
 2   in_reply_to_user_id         78 non-null     float64            
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         181 non-null    float64            
 7   retweeted_status_user_id    181 non-null    float64            
 8   retweeted_status_timestamp  181 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

### Issue #2:

#### Define

- Drop columns with a lot of missing values.

#### Code

In [21]:
# Drop cols with over 90% null values
archive_clean.drop(columns=['in_reply_to_status_id','in_reply_to_user_id',\
                            'retweeted_status_id','retweeted_status_user_id',\
                            'retweeted_status_timestamp' ], inplace=True)

#### Test

In [22]:
# See changes
archive_clean.head(1)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


### Issue #3:

#### Define
- Change invalid dog names in the `archive` `name` column to None.



#### Code

In [23]:
# Replace invalid dog names with None
archive_clean['name'] = archive_clean.name.str.replace('\d+|^[a-z]', 'None', regex=True)

### Test

In [24]:
# See Changes
archive_clean.name.value_counts()

None          800
Charlie        12
Oliver         11
Lucy           11
Cooper         11
             ... 
Dex             1
Ace             1
Tayzie          1
Grizzie         1
Christoper      1
Name: name, Length: 955, dtype: int64

In [25]:
archive_clean.name[275]

'None'

### Issue#4:



#### Define

- Extract `doggo`,`floofer`,`pupper`,`puppo` from `text` col.


#### Code

In [26]:
# Extract dog types from text col and drop type cols
archive_clean['dog_type'] = archive_clean.text.str.extract('(floofer|doggo|pupper|puppo)', expand=True)
archive_clean.drop(columns=['doggo', 'floofer', 'pupper', 'puppo'], inplace=True)

#### Test

In [27]:
# See Changes
archive_clean.head(2)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type
0,892420643555336193,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN
1,892177421306343426,2017-08-01 00:17:27+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN


In [28]:
archive_clean.dog_type.value_counts()

pupper     265
doggo       93
puppo       37
floofer      4
Name: dog_type, dtype: int64

### Issue#5:


#### Define

- Extract only the text containing the source of the tweet and the source url.


#### Code

In [29]:
# Create source_text and source_urls from source col
archive_clean['source_text'] = archive_clean.source.str.extract('>([A-Za-z].+)<')[0]
archive_clean['source_url'] = archive_clean.source.str.extract('([a-z]+://.+)("\s)')[0]
# Drop source col
archive_clean.drop('source', axis=1,inplace=True)

#### Test

In [30]:
# See Changes
archive_clean.head()

,tweet_id,timestamp,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type,source_text,source_url
0,892420643555336193,2017-08-01 16:23:56+00:00,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN,Twitter for iPhone,http://twitter.com/download/iphone
1,892177421306343426,2017-08-01 00:17:27+00:00,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN,Twitter for iPhone,http://twitter.com/download/iphone
2,891815181378084864,2017-07-31 00:18:03+00:00,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,NaN,Twitter for iPhone,http://twitter.com/download/iphone
3,891689557279858688,2017-07-30 15:58:51+00:00,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,NaN,Twitter for iPhone,http://twitter.com/download/iphone
4,891327558926688256,2017-07-29 16:00:24+00:00,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,NaN,Twitter for iPhone,http://twitter.com/download/iphone


### Issue#6:


#### Define

- Rename the expanded_urls column to tweet_url since it points to the specific tweet.


#### Code

In [31]:
# Rename expanded_urls col
archive_clean.rename(columns={'expanded_urls': 'tweet_url'}, inplace=True)

#### Test

In [32]:
# See Changes
archive_clean.head(1)

,tweet_id,timestamp,text,tweet_url,rating_numerator,rating_denominator,name,dog_type,source_text,source_url
0,892420643555336193,2017-08-01 16:23:56+00:00,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN,Twitter for iPhone,http://twitter.com/download/iphone


### Issue#7:

#### Define

- Change all `tweet_id` column datatypes from `int` type to `str` type.



#### Code

In [33]:
# Change all tweet_id datatypes to str type
archive_clean['tweet_id'] = archive_clean.tweet_id.astype(str)
predictions_clean['tweet_id'] = predictions_clean.tweet_id.astype(str)
tweet_json_clean['tweet_id'] = tweet_json_clean.tweet_id.astype(str)

#### Test

In [34]:
# See Changes
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2356 non-null   object             
 1   timestamp           2356 non-null   datetime64[ns, UTC]
 2   text                2356 non-null   object             
 3   tweet_url           2297 non-null   object             
 4   rating_numerator    2356 non-null   int64              
 5   rating_denominator  2356 non-null   int64              
 6   name                2356 non-null   object             
 7   dog_type            399 non-null    object             
 8   source_text         2356 non-null   object             
 9   source_url          2356 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(2), object(7)
memory usage: 184.2+ KB


In [35]:
# See Changes
predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   object 
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


In [36]:
# See Changes
tweet_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2327 entries, 0 to 2326
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   tweet_id       2327 non-null   object             
 1   timestamp      2327 non-null   datetime64[ns, UTC]
 2   retweet_count  2327 non-null   int64              
 3   reply_count    2327 non-null   int64              
 4   like_count     2327 non-null   int64              
dtypes: datetime64[ns, UTC](1), int64(3), object(1)
memory usage: 91.0+ KB


### Issue#8:

#### Define

- Merge archive and tweet_json dataframes.



#### Code

In [42]:
# Merge archive and tweet_json dataframes
df_merge = pd.merge(archive_clean,
                    tweet_json_clean.drop('timestamp',axis=1), on='tweet_id')

#### Test

In [43]:
# See Changes
df_merge.head(2)

,tweet_id,timestamp,text,tweet_url,rating_numerator,rating_denominator,name,dog_type,source_text,source_url,retweet_count,reply_count,like_count
0,892420643555336193,2017-08-01 16:23:56+00:00,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN,Twitter for iPhone,http://twitter.com/download/iphone,6969,139,33696
1,892177421306343426,2017-08-01 00:17:27+00:00,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN,Twitter for iPhone,http://twitter.com/download/iphone,5272,170,29220


## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [44]:
# Merge all dataframes and save to csv
master_df = pd.merge(df_merge, predictions_clean, on='tweet_id')

In [48]:
# See number of observations
master_df.shape

(2057, 24)

In [49]:
# Save master dataframe to a csv file.
master_df.to_csv('twitter_archive_master.csv', index=False)

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

In [88]:
# Read the master csv file into a pandas dataframe
df = pd.read_csv('twitter_archive_master.csv')
df.head(2)

,tweet_id,timestamp,text,tweet_url,rating_numerator,rating_denominator,name,dog_type,source_text,source_url,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,2017-08-01 16:23:56+00:00,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN,Twitter for iPhone,http://twitter.com/download/iphone,...,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,2017-08-01 00:17:27+00:00,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN,Twitter for iPhone,http://twitter.com/download/iphone,...,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


In [71]:
# See df summary info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2057 entries, 0 to 2056
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            2057 non-null   int64  
 1   timestamp           2057 non-null   object 
 2   text                2057 non-null   object 
 3   tweet_url           2057 non-null   object 
 4   rating_numerator    2057 non-null   int64  
 5   rating_denominator  2057 non-null   int64  
 6   name                2057 non-null   object 
 7   dog_type            334 non-null    object 
 8   source_text         2057 non-null   object 
 9   source_url          2057 non-null   object 
 10  retweet_count       2057 non-null   int64  
 11  reply_count         2057 non-null   int64  
 12  like_count          2057 non-null   int64  
 13  jpg_url             2057 non-null   object 
 14  img_num             2057 non-null   int64  
 15  p1                  2057 non-null   object 
 16  p1_con

In [126]:
# Get dogs classified as dogs in all probability levels
dog_class = df.query('p1_dog == True & p2_dog == True & p3_dog == True')

In [127]:
# See dog_class sample size
dog_class.shape

(1232, 24)

In [155]:
# Get top 10 dogs with a high probability 1 
top_10 = dog_class.groupby(['tweet_id', 'p1']).p1_conf.nlargest(1).sort_values(ascending=False)[:10]
top_10

tweet_id            p1                      
697463031882764288  Labrador_retriever  1192    0.999885
675149409102012420  chow                1581    0.999876
709566166965075968  chow                1043    0.999837
814986499976527872  dalmatian           376     0.999828
777621514455814149  chow                606     0.999823
680115823365742593  pug                 1446    0.999365
697259378236399616  Great_Dane          1194    0.999223
886366144734445568  French_bulldog      28      0.999201
872820683541237760  pug                 84      0.999120
667435689202614272  Rottweiler          1980    0.999091
Name: p1_conf, dtype: float64

In [156]:
# Get bottom 10 dogs with a low probability 1
bottom_10 = dog_class.groupby(['tweet_id', 'p1']).p1_conf.nlargest(1).sort_values(ascending=False)[-10:]
bottom_10

tweet_id            p1                      
880095782870896641  miniature_pinscher  56      0.120298
674793399141146624  giant_schnauzer     1596    0.119693
760893934457552897  Blenheim_spaniel    701     0.113992
693642232151285760  Scottish_deerhound  1226    0.111893
747512671126323200  Cardigan            812     0.111493
698262614669991936  Italian_greyhound   1181    0.107948
686683045143953408  Norwich_terrier     1326    0.100499
683773439333797890  miniature_pinscher  1377    0.072885
668507509523615744  basenji             1920    0.055379
666644823164719104  Ibizan_hound        2018    0.044333
Name: p1_conf, dtype: float64

In [160]:
# Get top 5 highly rated dogs
top_rated = df.groupby(['tweet_id','dog_type']).rating_numerator.nlargest(1).sort_values(ascending=False)[:5]
top_rated

tweet_id            dog_type      
731156023742988288  doggo     909     204
684225744407494656  pupper    1364    143
684222868335505415  pupper    1365    121
713900603437621249  pupper    1007     99
675853064436391936  pupper    1555     88
Name: rating_numerator, dtype: int64

In [153]:
# Get top 5 most retweeted dog types
retweet = df.groupby(['tweet_id','dog_type']).retweet_count.nlargest(1).sort_values(ascending=False)[:5]
retweet

tweet_id            dog_type     
744234799360020481  doggo     836    70330
739238157791694849  doggo     871    52631
822872901745569793  puppo     319    39751
866450705531457537  pupper    110    30055
795464331001561088  doggo     488    21866
825535076884762624  puppo     305    16300
Name: retweet_count, dtype: int64

In [124]:
# Get top 5 most engaged dog type tweets
engagement = df.groupby(['tweet_id','dog_type']).reply_count.nlargest(1).sort_values(ascending=False)[:5]
engagement

In [125]:
# Get top 5 most liked dog type tweets
favourite = df.groupby(['tweet_id','dog_type']).like_count.nlargest(1).sort_values(ascending=False)[:5]

In [130]:
# Get all dogs not classified as dogs in all probability levels
not_dogs = df.query('p1_dog == False & p2_dog == False & p3_dog == False')

In [131]:
not_dogs.shape

(321, 24)

In [166]:
# Get the dog type most wrongly classified and what it was classified as
not_dogs.groupby(['p1', 'p2', 'p3']).dog_type.value_counts().nlargest(1)

p1         p2           p3           dog_type
mousetrap  black_widow  paddlewheel  pupper      2
Name: dog_type, dtype: int64

In [169]:
wrong = not_dogs.dog_type.value_counts()
wrong

pupper    42
doggo     11
puppo      1
Name: dog_type, dtype: int64

### Insights:

1. Top 5 dog breeds correctly classified by the algorithm. 

2. Top tweet sources.
 
3. Top rated dog type and breed.

4. Top 5 popular dog names.

5. Top 5 most retweeted and dog breed.

6. Top 5 most liked and dog breed.

7. Top 5 most engaged tweets.

### Visualization